In [ ]:
%pip install openai

In [ ]:
%pip install serpapi

In [ ]:
%pip install python-dotenv

In [ ]:
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
import os 

openai_api_key=os.getenv("OPENAI_API_KEY")
serpapi_key=os.getenv("SERP_API_KEY")



In [ ]:
MODEL = "gpt-4o-mini"

# DEFINITION OF TOOLS THAT WILL BE  USED BY LLM

In [ ]:
# The particular dictionary structure that's required to describe our function for the LLM

flight_function = {
    "name": "get_flight_info",
    "description": "Get real-time information about a specific flight. Call this whenever you need to know the ticket price, for example, when a customer asks 'How much is a ticket to this city?'.\
    Call this whenever you need to know the flight details such as departure time, arrival time, available flights, flight route, duration of a flight, flight type, layovers, travel class, airplane, legroom, and extensions.",
    "parameters": {
        "type": "object",
        "properties": {
            "departure": {
                "type": "string",
                "description": "The IATA code of the airport the customer wants to travel from. For example, 'AUS' for Austin-Bergstrom International Airport.",
            },
            "destination": {
                "type": "string",
                "description": "The IATA code of the airport the customer wants to travel to. For example, 'JFK' for John F. Kennedy International Airport.",
            },
            "departure_date": {
                "type": "string",
                "format": "date",
                "description": "The date of departure in YYYY-MM-DD format. For example, '2024-06-15'.",
            },
            "return_date": {
                "type": "string",
                "format": "date",
                "description": "The return date in YYYY-MM-DD format for round trips. This field is optional.",
            },
    },
        "required": ["departure", "destination", "departure_date","return_date"],
        "additionalProperties": False,
}
}   


In [ ]:
tools = [
    {"type": "function", "function": flight_function}
]


# UTILITIES

In [ ]:
from datetime import datetime

system_message = "You are a helpful personal travel assistant"
system_message += "Help the user determine the experience they want to achieve from their travel."
system_message += "Suggest locations where they can have their desired experience."
system_message += "Provide real-time cost to travel."
system_message += "Provide guidance on how to travel to the desired location."
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "You should always be polite."
system_message += "Start a conversation by introducing yourself for example: 'Hi, I am Amy, your personal travel assistant.How may I help you?'."
system_message += f"Today's date is {datetime.today()}"

In [ ]:
import json

In [ ]:
from datetime import datetime

In [ ]:
def format_flight_data(flight_data):
    flights = flight_data.get("best_flights", [])
    if not flights:
        return "❌ No flights found. Try another date or route."

    all_flight_details = []
    
    for number, flight in enumerate(flights):
        flight_details = []
        price = f"💰 **Price**: ${flight['price']}\n"
        total_duration = f"⌛ **Total Flight Duration**: {flight['total_duration']} minutes\n"
        flight_type = f"🛫 **Type**: {flight['type']}\n"
        #layovers = f"🛬 **Layovers**: {flight['layovers']}\n"

        flight_details.append(price)
        flight_details.append(total_duration)
        flight_details.append(flight_type)
        #flight_details.append(layovers)

        sub_flights = flight['flights']
        for sub_flight in sub_flights:
            details = (
                f"✈️ **{sub_flight['airline']} {sub_flight['flight_number']}**\n"
                f"🛫 **Departure**: {sub_flight['departure_airport']['name']} ({sub_flight['departure_airport']['id']}) "
                f"at {sub_flight['departure_airport']['time']}\n"
                f"🛬 **Arrival**: {sub_flight['arrival_airport']['name']} ({sub_flight['arrival_airport']['id']}) "
                f"at {sub_flight['arrival_airport']['time']}\n"
                f"⌛ **Duration**: {sub_flight['duration']} minutes\n"
                f"💺 **Class**: {sub_flight['travel_class']}\n"
                f"✈️ **Aircraft**: {sub_flight['airplane']}\n"
                f"🌎 **Extras**: {', '.join(sub_flight['extensions']) if sub_flight['extensions'] else 'None'}\n"
                f"----------------------------"
            )
            flight_details.append(details)
            
        all_flight_details.append(f"✈️ Flight Option {number + 1}:\n" + "\n".join(flight_details))
    
    return "\n\n".join(all_flight_details)


Reference link for SERAPI: https://pypi.org/project/serpapi/
Reference link for search engine: https://serpapi.com/google-flights-api

In [ ]:

from serpapi import search
def get_flight_info(departure, arrival, departure_date, return_date):
    params = {
      "api_key": serpapi_key,
      "engine": "google_flights",
      "hl": "en",
      "gl": "us",
      "departure_id": departure,
      "arrival_id": arrival,
      "outbound_date": departure_date,
      "return_date": return_date,
      "currency": "USD"
    }
    # prevent past date from being sent to the api
    if datetime.strptime(departure_date, '%Y-%m-%d') < datetime.today():
        print("step1 failed")
        return []
    elif datetime.strptime(return_date, '%Y-%m-%d') < datetime.today():
        print("step2 failed")
        return []
    elif datetime.strptime(departure_date, '%Y-%m-%d') > datetime.strptime(return_date, '%Y-%m-%d'):
        print("step3 failed")
        return []
    else:
        flight_data = search(params)
        flight_details = format_flight_data(flight_data)
        return flight_details

In [ ]:
def handle_tool_call(arguments, name, tool_call):
    if name == "get_flight_info":
        departure_code = arguments.get('departure')
        destination_code = arguments.get('destination')
        departure_date = arguments.get('departure_date')
        return_date = arguments.get('return_date')
        
        flight_details = get_flight_info(departure_code, destination_code, departure_date, return_date)
        response = {
            "role": "tool",
            "name": name,
            "content": json.dumps({"departure": departure_code, "destination": destination_code,\
                                   "departure_date":departure_date, "return_date": return_date,\
                                  "flight_details": flight_details}),
            "tool_call_id": tool_call.id
        }
        
    else:
        print(f"Error: Unrecognized tool name '{name}'")
        return None  
    return response

Reference Links for instantiating OpenAI: https://platform.openai.com/docs/api-reference/chat

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)


def chat(message):

    messages = [{"role": "system", "content": system_message},
                {"role": "user", "content": message}
               ]

    response = client.chat.completions.create(
        model=MODEL, 
        messages=messages, 
        tools=tools)
    
    
    finalresponse=None
    print(f"response: {response}\n")

    print(f"reason: {response.choices[0].finish_reason}\n")
    print(f"message: {response.choices[0].message}\n")
    print(f"content: {response.choices[0].message.content}\n")
   
    if response.choices[0].finish_reason=="tool_calls":
        # get the name of tool called
        message = response.choices[0].message
        tool_call = message.tool_calls[0]
        print(f"arguments: {json.loads(tool_call.function.arguments)}")
        arguments = json.loads(tool_call.function.arguments)
        tool_name = tool_call.function.name
        response = handle_tool_call(arguments, tool_name, tool_call)
        if tool_name == "get_flight_info":
            messages.append(message)
            messages.append(response)
        response = client.chat.completions.create(model=MODEL, messages=messages)
    reply = response.choices[0].message.content
    finalresponse = [{"role": "assistant", "content":reply}]
    
    return finalresponse

In [ ]:
message ="Find me tickets from Bangalore on the 10th March, 2025 to Pantnagar on 12thmarch,2025"

In [ ]:
travel_agents_response=chat(message)

In [ ]:
print(travel_agents_response)

In [ ]:
#TODO
#1. Build a User Interface with gradio
#2. Deploy the model on Heroku
#3. Test the model on the deployed server
#4. Update the documentation
#5. Add history to the chat